# [Supernovae Data by Astrocatalogs](https://github.com/astrocatalogs/sne-2015-2019)

## Data Description and Exploratory Data Analysis

The data with which we are going to populate our Data Lake is with the **Open Supernovae Catalog (OSC)**. We will use a collection of all discovered supernovae from 2015 to 2019. This repository contains a variety of information about its discovery and physical attributes. For a deeper understanding, see its **[Schema's information](https://github.com/astrocatalogs/schema)**. The data is stored in .JSON files and each file refers to a different supernovae.

First thing first, let us clone the repository inside our data folder. From the main directory, go to data and git clone the following repository there: https://github.com/astrocatalogs/sne-2015-2019.git.

This should create a folder filled with .JSON files called **sne-2015-2019**. Be carefull when managing these files inside Jupyter Lab, this folder is a **CHONK BOY**! **ʕ•ᴥ•ʔ**

Now, we will import a single supernovae data to see what we are working with.

In [5]:
import pandas as pd
import json

In [12]:
f = r'../data/sne-2015-2019/ASASSN-15ab.json'

with open(f, 'r') as j:
    raw = json.loads(j.read())

The first key is only the supernovae's name. This could be handfull in case we want to merge every .JSON file into only one gigantic .JSON.

In [13]:
print(raw.keys())
print(list(raw.keys())[0])

dict_keys(['ASASSN-15ab'])
ASASSN-15ab


Once we open the first key, we begin to see every possible information collected from these supernovae. This is where we start to imagine what data structure will correctly fit our needs. Here we stumble on a couple of problems. Firstly, these measurement can have multiple values from differents observations. For instance, the **dec** key has two possible values from distinct sources. Secondly, by looking at our [Schema](https://github.com/astrocatalogs/schema) we see multiple attributes that are not present in the supernovae we are analysing. This could be a barrier when recursively creating a **DataFrame**.

In [14]:
print(raw['ASASSN-15ab'].keys())

dict_keys(['schema', 'name', 'sources', 'alias', 'claimedtype', 'comovingdist', 'dec', 'discoverdate', 'discoverer', 'ebv', 'host', 'hostdec', 'hostoffsetang', 'hostoffsetdist', 'hostra', 'lumdist', 'maxabsmag', 'maxappmag', 'maxdate', 'maxvisualabsmag', 'maxvisualappmag', 'maxvisualdate', 'ra', 'redshift', 'velocity', 'photometry'])


In [15]:
for key in raw['ASASSN-15ab']:
    value = raw['ASASSN-15ab'][key]
    # if isinstance(value, list):
    #      value = value[0]
    # else:
    #      value = value
    print(f'{key} : {value}\n\n')

schema : https://github.com/astrocatalogs/supernovae/blob/d3ef5fc/SCHEMA.md


name : ASASSN-15ab


sources : [{'name': '2016A&A...594A..13P', 'bibcode': '2016A&A...594A..13P', 'reference': 'Planck Collaboration et al. (2016)', 'alias': '1'}, {'name': '2015ATel.6864....1D', 'bibcode': '2015ATel.6864....1D', 'reference': 'Dong et al. (2015)', 'alias': '2'}, {'name': '2015ATel.6882....1S', 'bibcode': '2015ATel.6882....1S', 'reference': 'Shappee et al. (2015)', 'alias': '3'}, {'name': '2011ApJ...737..103S', 'bibcode': '2011ApJ...737..103S', 'reference': 'Schlafly & Finkbeiner (2011)', 'alias': '4'}, {'name': 'ASAS-SN Supernovae', 'url': 'http://www.astronomy.ohio-state.edu/~assassin/sn_list.html', 'alias': '5'}, {'name': 'Latest Supernovae', 'secondary': True, 'url': 'http://www.rochesterastronomy.org/snimages/snredshiftall.html', 'alias': '6'}, {'name': 'The Open Supernova Catalog', 'bibcode': '2017ApJ...835...64G', 'reference': 'Guillochon et al. (2017)', 'secondary': True, 'url': 'https

By looking at the data from multiple supernovae, we've figured that the best way to build a database is to slip our data into a relational and a non-relational set. For instance, the following fields are well behaved and could be structured as a relational database.

```
alias, claimedtype, comovingdist, dec, discoverdate, discoverer, ebv, host, hostdec, hostoffsetang, hostoffsetdist, hostra, lumdist, maxabsmag, maxappmag, maxdate, maxvisualabsmag, maxvisualappmag, maxvisualdate, ra, redshift, velocity
```

Even if some of them have two distinct values - e.g., ```dec``` and ```ra``` - we will pick the first one for simplicity.

On the contrary, fields such as ```photometry, spectra, X-ray,``` and  ```radio``` are multidimensional with a single key (supernovae's name). Therefore, they are candidates to their own non-relational sub database, where they are indexed with the supernovae's name. We are going to use **AWS Redshift** to create the relational database, and **MongoDB** for the .JSON based database.

## Relational Data Transformation

Throughout the beginning of this course, we will only use the relational part of this data. Therefore, the first step is to create a function capable of turning all these values into columns with their respective data. Following the documentation from **Pandas** on the function ```read_json``` [[DOC]](https://pandas.pydata.org/pandas-docs/version/1.1.2/reference/api/pandas.read_json.html), we are going to create a list like object with the pattern

```
[{column -> value}, ... , {column -> value}] BUT ACTUALLY TBD
```

In [60]:
def linearization(raw: dict, non_relational: list) -> pd.DataFrame:
    
    
    prime = list(raw.keys())[0]
    idx = 1
    index = {}
    
    for key in raw[prime]:
        temp = {}
        
        if key in non_relational:
            pass
        else:
            value = raw[prime][key]
            
            if isinstance(value, list):
                value = value[0]
                
                for sec_key in value:
                    temp[key + '_' + sec_key] = value[sec_key]
                    print(f'temp : {temp}\n')
                    index[idx] = temp
                    print(f'index : {index}\n')
    idx += 1
                    
    return pd.DataFrame.from_dict(index, orient = 'index')
        

In [61]:
df = linearization(raw, ['photometry', 'source'])

temp : {'sources_name': '2016A&A...594A..13P'}

index : {1: {'sources_name': '2016A&A...594A..13P'}}

temp : {'sources_name': '2016A&A...594A..13P', 'sources_bibcode': '2016A&A...594A..13P'}

index : {1: {'sources_name': '2016A&A...594A..13P', 'sources_bibcode': '2016A&A...594A..13P'}}

temp : {'sources_name': '2016A&A...594A..13P', 'sources_bibcode': '2016A&A...594A..13P', 'sources_reference': 'Planck Collaboration et al. (2016)'}

index : {1: {'sources_name': '2016A&A...594A..13P', 'sources_bibcode': '2016A&A...594A..13P', 'sources_reference': 'Planck Collaboration et al. (2016)'}}

temp : {'sources_name': '2016A&A...594A..13P', 'sources_bibcode': '2016A&A...594A..13P', 'sources_reference': 'Planck Collaboration et al. (2016)', 'sources_alias': '1'}

index : {1: {'sources_name': '2016A&A...594A..13P', 'sources_bibcode': '2016A&A...594A..13P', 'sources_reference': 'Planck Collaboration et al. (2016)', 'sources_alias': '1'}}

temp : {'alias_value': 'ASASSN-15ab'}

index : {1: {'alias_v

In [57]:
df

,sources_name,sources_bibcode,sources_reference,sources_alias,alias_value,alias_source,claimedtype_value,claimedtype_source,comovingdist_value,comovingdist_derived,...,maxvisualdate_derived,maxvisualdate_source,ra_value,ra_u_value,ra_source,redshift_value,redshift_source,velocity_value,velocity_u_value,velocity_source
1,2016A&A...594A..13P,2016A&A...594A..13P,Planck Collaboration et al. (2016),1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,ASASSN-15ab,"2,5,6,8",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,IIn,"2,3,5,6,8",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.5,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
